In [1]:
import faiss
from sentence_transformers import SentenceTransformer
import pickle
import torch

/Users/artashesmezhlumyan/Desktop/capstone/capstonevenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

/Users/artashesmezhlumyan/Desktop/capstone/capstonevenv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## L2

In [3]:
import faiss
import numpy as np

def search_nearest_neighbors(query, model, index, doc_list, k=5):

    query_embedding = model.encode([query]) 
    if len(query_embedding.shape) == 1:
        query_embedding = np.expand_dims(query_embedding, axis=0)
    distances, indices = index.search(query_embedding, k)
    nearest_neighbors = {
        'sentences': [doc_list[idx] for idx in indices[0]],
        'distances': distances[0]
    }

    return nearest_neighbors

with open("../dataset/embeddings.pkl", "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data["sentences"]
    stored_embeddings = stored_data["embeddings"]

if isinstance(stored_embeddings, torch.Tensor):
    stored_embeddings = stored_embeddings.cpu().numpy()  
    
dimension = stored_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(stored_embeddings)  

query = "What is the level of agreement between the fully differential calculation in perturbative quantum chromodynamics for the production of massive photon pairs and data from the Fermilab Tevatron, and what predictions are made for more detailed tests with CDF and DO data"
nearest_neighbors = search_nearest_neighbors(query, model, index, stored_sentences, k=5)

print("Nearest neighbors and their distances:")
for sentence, distance in zip(nearest_neighbors['sentences'], nearest_neighbors['distances']):
    print(f"Sentence: {sentence}, Distance: {distance:.4f}")


Nearest neighbors and their distances:
Sentence:   A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
that enhanced sensitivity to the signal can be obtained with